## PROBLEM NEEDS TO BE FIXED

--> until SCAN_30 it works properly but after that it mismatches the labels and id with the map file


## IMPORTING LIBRARIES

In [1]:
import open3d as o3d
import numpy as np
import pandas as pd
import os
import sys
from plyfile import PlyData, PlyElement

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# DATA LOADING AND INITIAL VISUALIZATION

## CONFIGURATION

In [55]:

# --- Configuration ---
# IMPORTANT: Adjust these paths to your downloaded EDF dataset location
dataset_root_path = 'C:/RUTUL/GBC/WIP/Dataset/EDF Industrial Facility'

# Assuming these file names based on your description
global_labels_file = 'ytrain_i9bpfD4.csv' # The big Excel file with all ID-class mappings
train_map_file = 'suppfiles_HtLAvex/ytrain_map_ind_station.csv' # Mapping for training stations
test_map_file = 'suppfiles_HtLAvex/ytest_map_ind_station.csv'   # Mapping for testing stations

# Choose whether to load from 'train' or 'test' set
data_split = 'train' # or 'test'

# Choose which station ID to visualize (e.g., 0, 1, 31, 32 based on your map files)
# If data_split is 'train', choose a station_id from ytrain_map_ind_station.csv
# If data_split is 'test', choose a station_id from ytest_map_ind_station.csv
station_id_to_visualize = 4 # Example: visualizing SCAN_0.ply from train set


## Defining Path

In [36]:

# --- Define Paths ---
global_labels_path = os.path.join(dataset_root_path, global_labels_file)

if data_split == 'train':
    map_file_path = os.path.join(dataset_root_path, train_map_file)
    ply_folder_path = os.path.join(dataset_root_path, 'xtrain_kW4SLO1') # Assuming 'train' subdir for PLY files
elif data_split == 'test':
    map_file_path = os.path.join(dataset_root_path, test_map_file)
    ply_folder_path = os.path.join(dataset_root_path, 'test') # Assuming 'test' subdir for PLY files
else:
    raise ValueError("data_split must be 'train' or 'test'")

ply_file_name = f'SCAN_{station_id_to_visualize}.ply'
ply_file_path = os.path.join(ply_folder_path, ply_file_name)

# Define class colors for visualization (you can expand this for all 10 classes)
# These are just example colors. You can choose more distinct ones.
# Make sure the index matches the label ID (e.g., colors[0] for label 0, colors[1] for label 1)
# These are the 10 classes as per EDF dataset documentation (0-9)
CLASS_COLORS = {
    0: [0.5, 0.5, 0.5],  # Background (grey)
    1: [1.0, 0.0, 0.0],  # Beams (red)
    2: [0.0, 1.0, 0.0],  # Cabletrays (green)
    3: [0.0, 0.0, 1.0],  # Civils (blue - for walls/floors)
    4: [1.0, 1.0, 0.0],  # Gratings (yellow)
    5: [1.0, 0.5, 0.0],  # Guardrails (orange)
    6: [0.0, 1.0, 1.0],  # Hvac (cyan)
    7: [0.5, 0.0, 0.5],  # Ladders (purple)
    8: [1.0, 0.0, 1.0],  # Pipping (magenta - for pipes!)
    9: [0.5, 0.5, 0.0],  # Supports (olive)
}


## Loading Global File

In [4]:

# --- 1. Load the Global Labels File ---
print(f"Loading global labels from: {global_labels_path}")
try:
    # --- FIX HERE: Remove header=None and names=['class'] ---
    # Pandas will now correctly read the header row and use 'class' as the column name.
    # We still specify dtype for the 'class' column to ensure it's loaded as int64.
    global_labels_df = pd.read_csv(global_labels_path, dtype={'class': np.int64})
    
    # Check if 'class' column exists AFTER reading with header
    if 'class' not in global_labels_df.columns:
        raise ValueError("Global labels CSV must contain a 'class' column after reading its header.")
    
    all_labels = global_labels_df['class'].values
    
    print(f"Successfully loaded {len(all_labels)} global labels. Data type: {all_labels.dtype}")
    print(f"DEBUG: First 5 labels: {all_labels[:5]}")
    # Ensure there are enough labels before trying to print last 5
    if len(all_labels) > 5:
        print(f"DEBUG: Last 5 labels: {all_labels[-5:]}")
    
except MemoryError as e:
    print(f"ERROR: Not enough memory to load '{global_labels_file}' (1.5GB). Try increasing RAM or using a server.")
    sys.exit(1)
except Exception as e:
    print(f"Error loading global labels CSV: {e}. Please check '{global_labels_file}' and its content/format.")
    sys.exit(1)


Loading global labels from: C:/RUTUL/GBC/WIP/Dataset/EDF Industrial Facility\ytrain_i9bpfD4.csv
Successfully loaded 130661990 global labels. Data type: int64
DEBUG: First 5 labels: [8 8 8 8 8]
DEBUG: Last 5 labels: [8 8 3 8 9]


## Loading Map FIle

In [5]:

# --- 2. Load the Map File for the chosen split ---
print(f"Loading map file from: {map_file_path}")
try:
    map_df = pd.read_csv(map_file_path, header=None, names=['Station_index', 'index_start', 'index_end'],
                         dtype={'Station_index': np.int64, 'index_start': np.int64, 'index_end': np.int64})    # Find the row corresponding to our chosen station_id
    station_info = map_df[map_df['Station_index'] == station_id_to_visualize]

    if station_info.empty:
        print(f"ERROR: Station ID {station_id_to_visualize} not found in map file: {map_file_path}.")
        print(f"Available IDs in map file: {map_df['Station_index'].tolist()}")
        exit()
    index_start = station_info['index_start'].iloc[0]
    index_end = station_info['index_end'].iloc[0]
    print(f"Found mapping for Station {station_id_to_visualize}: start={index_start}, end={index_end}")
    print(f"DEBUG: Data types in map_df: \n{map_df.dtypes}")
    print(f"DEBUG: Map entry for Station {station_id_to_visualize}: start={index_start} (type: {type(index_start)}), end={index_end} (type: {type(index_end)})")
except Exception as e:
    print(f"Error loading map file or finding station info: {e}")
    print("Please check 'map_file_path' and 'station_id_to_visualize'.")
    exit()


Loading map file from: C:/RUTUL/GBC/WIP/Dataset/EDF Industrial Facility\suppfiles_HtLAvex/ytrain_map_ind_station.csv
Found mapping for Station 37: start=93562963, end=95226972
DEBUG: Data types in map_df: 
Station_index    int64
index_start      int64
index_end        int64
dtype: object
DEBUG: Map entry for Station 37: start=93562963 (type: <class 'numpy.int64'>), end=95226972 (type: <class 'numpy.int64'>)


In [6]:
try:
    # --- CRITICAL CHECKS FOR INDEX BOUNDS ---
    total_labels_count = len(all_labels)
    if index_start < 0:
        print(f"CRITICAL ERROR: 'index_start' ({index_start}) for Station {station_id_to_visualize} is negative.")
        sys.exit(1)
    
    if index_end >= total_labels_count:
        print(f"CRITICAL ERROR: 'index_end' ({index_end}) for Station {station_id_to_visualize} is OUT OF BOUNDS of 'all_labels' array (total size: {total_labels_count}).")
        print("This is the MOST LIKELY CAUSE of the '0 labels' or mismatch for later scans.")
        print("This means the map file has indices that point beyond the actual length of your global labels file.")
        print("Please verify the integrity of your map file ('ytrain_map_ind_station.csv') and confirm 'ytrain_i9bpfD4.csv' contains as many rows as 'index_end' suggests.")
        sys.exit(1)
        
    if index_end < index_start:
        print(f"CRITICAL ERROR: 'index_end' ({index_end}) is less than 'index_start' ({index_start}) for Station {station_id_to_visualize}. This would result in zero or negative labels.")
        sys.exit(1)
except Exception as e:
    print(f"Error loading map file or finding station info: {e}. Please check '{map_file_path}' and 'station_id_to_visualize'.")
    sys.exit(1)

## Label Extraction for the choosen station

In [ ]:

# --- 3. Extract Labels for the Specific Station ---
try:
    # NumPy slicing is [start:end] where 'end' is exclusive.
    # If index_end from CSV is inclusive, then we need index_end + 1.
    labels_for_station = all_labels[index_start : index_end + 1]
    
    if len(labels_for_station) == 0:
        print(f"CRITICAL ERROR: Extracted 0 labels for Station {station_id_to_visualize}. This means the slice {index_start}:{index_end+1} yielded no data.")
        print("This could be due to index_start == index_end + 1, or previous index_end out-of-bounds error not caught.")
        sys.exit(1)

    print(f"Extracted {len(labels_for_station)} labels for Station {station_id_to_visualize}.")
except IndexError as e:
    print(f"Index error when slicing global labels: {e}")
    print(f"This implies that the calculated slice [{index_start}:{index_end+1}] is out of bounds for 'all_labels' (total size: {len(all_labels)}).")
    print("This is highly likely the cause of your mismatch or '0 labels' issue for later scans.")
    print("Double-check if 'ytrain_i9bpfD4.csv' is fully loaded or if its actual row count matches the expected max 'index_end' from the map file.")
    sys.exit(1)



Extracted 1664010 labels for Station 37.


In [8]:
# --- 4. Load the Point Cloud (.ply file) for the specific station ---
print(f"\nAttempting to load PLY file: {ply_file_path}")

# --- DEBUGGING CHECKS ---
if not os.path.exists(ply_file_path):
    print(f"DEBUG: ERROR: PLY file DOES NOT EXIST at the specified path: {ply_file_path}")
    print("Please double-check your 'dataset_root_path', 'data_split' folder names (train/test), and 'station_id_to_visualize'.")
    sys.exit(1)
else:
    file_size = os.path.getsize(ply_file_path) / (1024 * 1024) # Size in MB
    print(f"DEBUG: PLY file exists. Size: {file_size:.2f} MB")
    if file_size < 0.001: # Very small size, might be genuinely empty or just a header
        print("DEBUG: WARNING: PLY file size is extremely small. It might be empty or corrupted.")

# --- MODIFIED PLY LOADING SECTION (using plyfile with correct element names) ---
try:
    print(f"Attempting to load PLY with 'plyfile' library for '{ply_file_name}' using custom element names...")
    plydata = PlyData.read(ply_file_path)

    # Extract points from the 'points' element
    if 'points' in plydata:
        points_element = plydata['points']
        points = np.vstack([points_element['x'], points_element['y'], points_element['z']]).T
        print(f"Successfully extracted {len(points)} points from 'points' element.")
    else:
        raise ValueError("PLY file does not contain a 'points' element as expected from header.")

    # Extract colors from the 'rgb' element
    colors = None
    if 'rgb' in plydata:
        rgb_element = plydata['rgb']
        # Assuming 'r', 'g', 'b' properties exist within 'rgb' element
        if 'r' in rgb_element.properties and 'g' in rgb_element.properties and 'b' in rgb_element.properties:
            colors = np.vstack([rgb_element['r'], rgb_element['g'], rgb_element['b']]).T
            # Normalize colors to [0, 1] if they are 0-255 (uchar)
            if colors.max() > 1.0:
                colors = colors / 255.0
            print("Successfully extracted RGB colors from 'rgb' element.")
        else:
            print("Warning: 'rgb' element found but missing 'r', 'g', or 'b' properties.")
    else:
        print("No 'rgb' element found in PLY file for colors.")

    # Optionally, extract intensity from the 'intensity' element
    intensity = None
    if 'intensity' in plydata:
        intensity_element = plydata['intensity']
        if 'i' in intensity_element.properties:
            intensity = intensity_element['i']
            print(f"Successfully extracted {len(intensity)} intensity values from 'intensity' element.")
        else:
            print("Warning: 'intensity' element found but missing 'i' property.")
    else:
        print("No 'intensity' element found in PLY file.")


    # Create Open3D PointCloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    if colors is not None:
        pcd.colors = o3d.utility.Vector3dVector(colors)
    # Note: Open3D does not have a direct 'intensity' property for visualization,
    # but you can store it or use it for other processing.

    if not pcd.has_points():
        raise ValueError("PointCloud created from plyfile has no points after conversion.")

except Exception as e:
    print(f"ERROR: Failed to load PLY file using 'plyfile' library with custom elements: {e}")
    print("This suggests a deeper issue with the PLY file's structure or corruption, or an unexpected header format.")
    sys.exit(1)

print(f"Successfully created Open3D PointCloud object with {len(pcd.points)} points.")



Attempting to load PLY file: C:/RUTUL/GBC/WIP/Dataset/EDF Industrial Facility\xtrain_kW4SLO1\SCAN_37.ply
DEBUG: PLY file exists. Size: 25.39 MB
Attempting to load PLY with 'plyfile' library for 'SCAN_37.ply' using custom element names...
Successfully extracted 1664010 points from 'points' element.
Successfully created Open3D PointCloud object with 1664010 points.


In [9]:
# --- 5. Verify Lengths and Prepare Colors for Visualization ---
print("\n--- Verifying Point Cloud and Label Lengths ---")
if len(pcd.points) != len(labels_for_station):
    print(f"\nCRITICAL ERROR: Mismatch detected! Number of points ({len(pcd.points)}) in '{ply_file_name}' "
          f"does NOT match number of labels ({len(labels_for_station)}) extracted from global file ({index_start}:{index_end+1}).")
    print("This confirms the problem. Check the previous error messages for root cause (likely index out of bounds).")
    print("Exiting as segmentation would be incorrect.")
    sys.exit(1) # Exit because the fundamental data integrity is broken
else:
    print(f"SUCCESS: Point cloud length ({len(pcd.points)}) matches label length ({len(labels_for_station)}).")

# Create an array to hold the colors based on labels
colors_from_labels = np.zeros((len(labels_for_station), 3))
for i, label_id in enumerate(labels_for_station):
    label_id_int = int(label_id)
    colors_from_labels[i] = CLASS_COLORS.get(label_id_int, [0.8, 0.8, 0.8])
pcd.colors = o3d.utility.Vector3dVector(colors_from_labels)
print("Point cloud colored by semantic labels.")

# --- 6. Visualize the Point Cloud ---
print(f"\nVisualizing Station {station_id_to_visualize} ({data_split} split). Close the visualization window to continue.")
o3d.visualization.draw_geometries([pcd])
print("Visualization complete.")



--- Verifying Point Cloud and Label Lengths ---
SUCCESS: Point cloud length (1664010) matches label length (1664010).
Point cloud colored by semantic labels.

Visualizing Station 37 (train split). Close the visualization window to continue.
Visualization complete.


In [63]:

# --- 7. Initial Exploration Insights ---
print("\n--- Initial Exploration Notes ---")
print(f"Number of points loaded for this station: {len(pcd.points)}")
unique_labels, counts = np.unique(labels_for_station, return_counts=True)
print("Label distribution for this station:")
label_names_map = {
    0: "Background", 1: "Beams", 2: "Cabletrays", 3: "Civils", 4: "Gratings",
    5: "Guardrails", 6: "Hvac", 7: "Ladders", 8: "Pipping", 9: "Supports"
}
for label, count in zip(unique_labels, counts):
    label_name = label_names_map.get(label, f"Unknown ({label})")
    print(f"  Class {label_name} (ID {label}): {count} points")


--- Initial Exploration Notes ---
Number of points loaded for this station: 1971824
Label distribution for this station:
  Class Background (ID 0): 46419 points
  Class Beams (ID 1): 46412 points
  Class Cabletrays (ID 2): 699 points
  Class Civils (ID 3): 1307478 points
  Class Gratings (ID 4): 218876 points
  Class Guardrails (ID 5): 98398 points
  Class Hvac (ID 6): 2187 points
  Class Ladders (ID 7): 221969 points
  Class Pipping (ID 8): 22888 points
  Class Supports (ID 9): 6498 points


In [90]:
# --- 5. Verify Lengths and Prepare Colors for Visualization ---
if len(pcd.points) != len(labels_for_station):
    print(f"\nCRITICAL WARNING: Number of points ({len(pcd.points)}) in '{ply_file_name}' "
          f"does NOT match number of labels ({len(labels_for_station)}) extracted via map file ({index_start}:{index_end+1}).")
    print("This is a common issue due to incorrect indexing, PLY file corruption, or mismatched data generation.")
    print("For visualization, truncating to the minimum length to avoid errors, but investigate this mismatch.")
    min_len = min(len(pcd.points), len(labels_for_station))
    pcd.points = o3d.utility.Vector3dVector(np.asarray(pcd.points)[:min_len])
    # Preserve original colors if they exist, otherwise set to dummy for slicing
    # Use the colors loaded from PLY, or if not available, default to grey.
    if pcd.has_colors():
        pcd.colors = o3d.utility.Vector3dVector(np.asarray(pcd.colors)[:min_len])
    else:
        pcd.colors = o3d.utility.Vector3dVector(np.full((min_len, 3), 0.7)) # Grey if no colors from PLY
    labels_for_station = labels_for_station[:min_len]
    print(f"Truncated to {min_len} points/labels for visualization.")

# Create an array to hold the colors based on labels
colors_from_labels = np.zeros((len(labels_for_station), 3))
for i, label_id in enumerate(labels_for_station):
    label_id_int = int(label_id) # Ensure label_id is an integer for dictionary lookup
    colors_from_labels[i] = CLASS_COLORS.get(label_id_int, [0.8, 0.8, 0.8]) # Default to light grey for unknown labels
pcd.colors = o3d.utility.Vector3dVector(colors_from_labels) # OVERWRITE with semantic colors
print("Point cloud colored by semantic labels.")

Point cloud colored by semantic labels.


In [91]:
# --- 6. Visualize the Point Cloud ---
print(f"\nVisualizing Station {station_id_to_visualize} ({data_split} split). Close the visualization window to continue.")

# Optional: Downsample for faster visualization if it's too dense
# pcd_downsampled = pcd.voxel_down_sample(voxel_size=0.05) # Adjust voxel_size as needed
# o3d.visualization.draw_geometries([pcd_downsampled])

o3d.visualization.draw_geometries([pcd])
print("Visualization complete.")




Visualizing Station 6 (train split). Close the visualization window to continue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
Visualization complete.


In [92]:
# --- 7. Initial Exploration Insights ---
print("\n--- Initial Exploration Notes ---")
print(f"Number of points loaded for this station: {len(pcd.points)}")
unique_labels, counts = np.unique(labels_for_station, return_counts=True)
print("Label distribution for this station:")
# Mapping label IDs to names for better readability
label_names_map = {
    0: "Background", 1: "Beams", 2: "Cabletrays", 3: "Civils", 4: "Gratings",
    5: "Guardrails", 6: "Hvac", 7: "Ladders", 8: "Pipping", 9: "Supports"
}
for label, count in zip(unique_labels, counts):
    label_name = label_names_map.get(label, f"Unknown ({label})")
    print(f"  Class {label_name} (ID {label}): {count} points")

print("\n**Key things to observe during visualization (if successful):**")
print("- **Geometric Integrity:** Does the point cloud look complete and well-aligned? Are there obvious holes or misalignments?")
print("- **Label Consistency:** Do the colored regions truly correspond to the object types? For example, are all pipes (magenta) connected correctly, and are walls (blue) forming continuous surfaces?")
print("- **Boundaries:** How clear are the boundaries between different segmented objects? Are they sharp or blurry?")
print("- **Object Sizes & Density:** Observe the varying sizes of objects (e.g., large walls vs. thin pipes) and the point density on different surfaces.")
print("- **Occlusions:** Identify areas where objects are hidden behind others. This affects how well algorithms can segment them.")
print("- **Noise & Outliers:** Look for isolated points or small clusters far from the main structures, which might be sensor noise.")
print("- **Overall Complexity:** Get a feel for the complexity of the industrial scene and the challenges it might pose for segmentation.")


--- Initial Exploration Notes ---
Number of points loaded for this station: 2920085
Label distribution for this station:
  Class Background (ID 0): 95715 points
  Class Beams (ID 1): 938 points
  Class Cabletrays (ID 2): 163731 points
  Class Civils (ID 3): 621962 points
  Class Gratings (ID 4): 1997 points
  Class Guardrails (ID 5): 71797 points
  Class Hvac (ID 6): 15866 points
  Class Ladders (ID 7): 808 points
  Class Pipping (ID 8): 1765894 points
  Class Supports (ID 9): 181377 points

**Key things to observe during visualization (if successful):**
- **Geometric Integrity:** Does the point cloud look complete and well-aligned? Are there obvious holes or misalignments?
- **Label Consistency:** Do the colored regions truly correspond to the object types? For example, are all pipes (magenta) connected correctly, and are walls (blue) forming continuous surfaces?
- **Boundaries:** How clear are the boundaries between different segmented objects? Are they sharp or blurry?
- **Object S

## Inital Insights

### SCAN0


Number of points loaded for this station: 2368290


Class Background (ID 0): 61829 points :: GREY

Class Beam (ID 1): 0 points :: Red

Class Cabletrays (ID 2): 39641 points :: GREEN

Class Civils (ID 3): 400446 points :: Blue(walls and floors)

Class Gratings (ID 4): 0 points :: Yellow

Class Guardrails (ID 5): 1403 points :: Orange

Class Hvac (ID 6): 2766 points :: Cyan

Class ladders (ID 7): 0 points :: purple

Class Pipping (ID 8): 1705937 points :: Magenta for pipes

Class Supports (ID 9): 156268 points :: olive


    0: [0.5, 0.5, 0.5],  # Background (grey)
    1: [1.0, 0.0, 0.0],  # Beams (red)
    2: [0.0, 1.0, 0.0],  # Cabletrays (green)
    3: [0.0, 0.0, 1.0],  # Civils (blue - for walls/floors)
    4: [1.0, 1.0, 0.0],  # Gratings (yellow)
    5: [1.0, 0.5, 0.0],  # Guardrails (orange)
    6: [0.0, 1.0, 1.0],  # Hvac (cyan)
    7: [0.5, 0.0, 0.5],  # Ladders (purple)
    8: [1.0, 0.0, 1.0],  # Pipping (magenta - for pipes!)
    9: [0.5, 0.5, 0.0],  # Supports (olive)

Label distribution for this station:: This is critical!

Look at the Class <Name> (ID <ID>): <Count> points for each unique label.

Identify Class Imbalance: You will likely see that some classes (e.g., 'Civils', 'Pipping') have many more points than others (e.g., 'Ladders', 'Gratings'). This class imbalance is a common challenge in segmentation and will heavily influence your model training strategy (e.g., requiring weighted loss functions, oversampling rare classes).
--> there is huge class imbalance like pipes are almost 1/2 of the total points whereas least are gaurdrails around 1k points

Confirm all expected classes are present: Are all 10 classes potentially represented in this scan, or just a subset?

--> not all classes have points there are there grating, beam and ladders which are not present


Identify Objects by Color: Based on your CLASS_COLORS map, try to visually confirm what each color represents. For SCAN_0.ply, can you clearly distinguish:

Magenta (Pipping): Do you see continuous pipe structures?
--> they are dense but they are not continous, huge gap are there in between

Blue (Civils): Are these mainly walls, floors, or large structural elements?
--> Yes, similary, some are desne whereas some are sparse

Green (Cabletrays): Can you pick out the cable trays?
--> yes, they are less but can be identified and those are dense and get not continous in between 
Red (Beams): Are there structural beams?
--> there is no points for that
Other Colors: What about Gratings, Guardrails, HVAC, Ladders, Supports, Background?


Boundary Observation: How clean are the boundaries between different colored segments? Are they sharp, or do they bleed into each other?
--> there are some overlaps other vise its clean and sharp. magenta and blue are overlapping somewhere and are less overlapped. olive and green are bit overlapping but i guess it should get some overlapped
Density & Completeness: Are there areas that are very dense or very sparse? Are there missing sections or large holes?
--> There are dense but are missing section 

Noise/Outliers: Do you see any isolated points floating away from the main structures that appear to be noise?
--> yes there are many sections which are far away and does not make any importance 

The visual is looked like it has been taken from one static Lidar sensor or scanner and then Point cloud was generated. beacause of that from one specific angle it has point cloud but looking from the opposite angle, the point cloud is missing but difficult to know exact which angle. for example a horizontal pipe has a dense( not so dense) point cloud but from opposite angle it has no points, looking like it has half pipe. 




### SCAN4

Number of points loaded for this station: 1654791

Label distribution for this station:

  Class Background (ID 0): 47192 points

  Class Cabletrays (ID 2): 41596 points

  Class Civils (ID 3): 318624 points

  Class Guardrails (ID 5): 209 points

  Class Hvac (ID 6): 122624 points

  Class Pipping (ID 8): 1050997 points

  Class Supports (ID 9): 73549 points

### SCAN8
Number of points loaded for this station: 3316643

Label distribution for this station:

  Class Background (ID 0): 820127 points

  Class Beams (ID 1): 2 points

  Class Cabletrays (ID 2): 3360 points

  Class Civils (ID 3): 801096 points

  Class Guardrails (ID 5): 179 points

  Class Hvac (ID 6): 56750 points

  Class Ladders (ID 7): 9 points

  Class Pipping (ID 8): 1549845 points

  Class Supports (ID 9): 85275 points

### SCAN6

Number of points loaded for this station: 2920085

Label distribution for this station:

  Class Background (ID 0): 95715 points

  Class Beams (ID 1): 938 points

  Class Cabletrays (ID 2): 163731 points

  Class Civils (ID 3): 621962 points

  Class Gratings (ID 4): 1997 points

  Class Guardrails (ID 5): 71797 points

  Class Hvac (ID 6): 15866 points

  Class Ladders (ID 7): 808 points

  Class Pipping (ID 8): 1765894 points

  Class Supports (ID 9): 181377 points

## Checking for the points in each file

In [18]:
import pandas as pd
import os

Suppfiles_path = 'C:/RUTUL/GBC/WIP/Dataset/EDF Industrial Facility/suppfiles_HtLAvex' # <<< VERIFY THIS PATH

train_map_file = 'ytrain_map_ind_station.csv'
test_map_file = 'ytest_map_ind_station.csv'

# Load train map
train_map_path = os.path.join(Suppfiles_path, train_map_file)
print(train_map_path)
train_map_df = pd.read_csv(train_map_path, header=None, names=['Station_index', 'index_start', 'index_end'])
train_map_df['point_count'] = train_map_df['index_end'] - train_map_df['index_start'] + 1
train_map_df['split'] = 'train'

# Load test map (if you have the test files available)
"""test_map_path = os.path.join(Suppfiles_path, test_map_file)
test_map_df = pd.read_csv(test_map_path, header=None, names=['Station_index', 'index_start', 'index_end'])
test_map_df['point_count'] = test_map_df['index_end'] - test_map_df['index_start'] + 1
test_map_df['split'] = 'test'"""

# Combine and sort for easy viewing
all_scans_df = train_map_df
all_scans_df_sorted = all_scans_df.sort_values(by='point_count').reset_index(drop=True)

print("--- Scan Sizes Overview ---")
print(f"Total scans: {len(all_scans_df_sorted)}")
print(f"Minimum points in a scan: {all_scans_df_sorted['point_count'].min()}")
print(f"Maximum points in a scan: {all_scans_df_sorted['point_count'].max()}")
print(f"Average points per scan: {all_scans_df_sorted['point_count'].mean():.2f}")
print(f"Median points per scan: {all_scans_df_sorted['point_count'].median()}")

print("\n--- Top 5 Smallest Scans ---")
print(all_scans_df_sorted.head(5))

print("\n--- Top 5 Largest Scans ---")
print(all_scans_df_sorted.tail(5))

print("\n--- Example of 'Average' Sized Scans (around median) ---")
median_count = all_scans_df_sorted['point_count'].median()
print(all_scans_df_sorted[(all_scans_df_sorted['point_count'] >= median_count * 0.9) &
                           (all_scans_df_sorted['point_count'] <= median_count * 1.1)].sample(n=min(5, len(all_scans_df_sorted))).sort_values(by='point_count'))

C:/RUTUL/GBC/WIP/Dataset/EDF Industrial Facility/suppfiles_HtLAvex\ytrain_map_ind_station.csv
--- Scan Sizes Overview ---
Total scans: 50
Minimum points in a scan: 1612043
Maximum points in a scan: 4132927
Average points per scan: 2613239.80
Median points per scan: 2656528.5

--- Top 5 Smallest Scans ---
   Station_index  index_start  index_end  point_count  split
0             45    109487941  111099983      1612043  train
1              4     10953766   12608556      1654791  train
2             37     93562963   95226972      1664010  train
3             42    104568399  106424116      1855718  train
4             58    143275610  145192546      1916937  train

--- Top 5 Largest Scans ---
    Station_index  index_start  index_end  point_count  split
45             64    160422335  163585348      3163014  train
46              8     21074972   24391614      3316643  train
47             12     32396605   35738625      3342021  train
48              2      4856610    8309083      3452

In [29]:
start = global_labels_df[global_labels_df["ID"]==index_start]
end = global_labels_df[global_labels_df["ID"]==index_end]
print(start.index)
print(end["ID"])


Index([83645062], dtype='int64')
85309071    95226972
Name: ID, dtype: int64


In [ ]:
labels_for_station_try = all_labels[start.index : end.index + 1]

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Index'

In [21]:
global_labels_df["ID"]==index_start

0            False
1            False
2            False
3            False
4            False
             ...  
130661985    False
130661986    False
130661987    False
130661988    False
130661989    False
Name: ID, Length: 130661990, dtype: bool

In [18]:
labels_for_station[:10]

array([3, 8, 3, 3, 9, 9, 3, 3, 8, 3])

In [17]:
for i in range(93562963,93562973):
    print(global_labels_df[global_labels_df["ID"]==i])
    

                ID  class
83645062  93562963      3
                ID  class
83645063  93562964      3
                ID  class
83645064  93562965      8
                ID  class
83645065  93562966      4
                ID  class
83645066  93562967      7
                ID  class
83645067  93562968      3
                ID  class
83645068  93562969      3
                ID  class
83645069  93562970      8
                ID  class
83645070  93562971      8
                ID  class
83645071  93562972      1


# DATA PREPROCESSING

### 1. Outlier Removal (Essential)
Problem Addressed: "Many sections which are far away and does not make any importance" (noise/outliers).

Method:

Statistical Outlier Removal (SOR): This is often the first step. It computes the average distance to k-nearest neighbors for each point. Points whose average distance is outside a certain standard deviation are removed.

Radius Outlier Removal: Removes points that have fewer than a specified number of neighbors within a given radius.

Why: Cleaning noise improves the quality of the point cloud, reduces irrelevant data, and can prevent the model from learning from spurious points.

### 2. Normalization & Centering (Standard Practice)
Problem Addressed: Preparing data for neural networks, which prefer inputs within a specific range.

Method:

Global Normalization: Translate all points so the centroid of the entire scene (or a bounding box center) is at the origin (0,0,0). Then, scale the points so they fit within a unit sphere or unit cube (e.g., [-1, 1] for each coordinate).

Why: Ensures consistency across different scans (which might have different absolute coordinates) and helps stabilize model training.

### 3. Downsampling (Crucial for Performance & Uniformity)
Problem Addressed:

Large number of points per scan (up to 3.3 million for SCAN8).

Potential for varying density (though you noted it's mostly consistent, downsampling can enforce uniformity).

Memory management for model training and web visualization.

Method:

Voxel Grid Downsampling: This is highly recommended. It divides the 3D space into a grid of voxels and, for each voxel, replaces all points within it with a single representative point (e.g., the centroid of the points in that voxel).

Why: Reduces the number of points while preserving the overall geometry and structure. It also creates a more uniform point density, which can benefit some point-based neural networks. You'll likely need different voxel sizes for training (e.g., 0.02m-0.05m) vs. web visualization (e.g., 0.1m-0.2m) to balance detail and performance.

### 4. Handling Class Imbalance (Critical for Model Performance)
Problem Addressed: Pipping and Civils dominate, while Guardrails, Hvac, Ladders, etc., are rare or absent in some scans. Model will be biased towards majority classes.

Method (for Training Data):

Weighted Cross-Entropy Loss: Assign higher weights to the loss contributions from minority classes during training. This makes the model "care more" about correctly classifying rare objects.

Oversampling Minority Classes: During data loading for training, you can oversample points belonging to rare classes (e.g., by duplicating them or using techniques like SMOTE for point clouds, though that's more complex).

Undersampling Majority Classes: Less common for point clouds due to information loss, but an option if memory is extremely constrained.

Class Grouping (Conditional): For classes that are always zero or extremely rare across all scans (e.g., Beams, Gratings, Ladders if they remain consistently absent or negligible), consider if they should be merged into a broader "Other Structural" or even "Background" class. However, since the dataset defines them, it's better to try to segment them if they appear in any scan.

Why: Prevents the model from simply predicting the majority class all the time and ensures it learns to distinguish rare but important components.

### 5. Data Augmentation (Essential for Robustness & Generalization)
Problem Addressed:

Static scanner limitations ("half-pipe" effect, missing views).

Improving model generalization to unseen variations in industrial scenes.

Method (Applied during Training):

Random Rotation: Rotate the point cloud around the Z-axis (up-axis) randomly. This helps the model learn features invariant to orientation.

Random Scaling: Slightly scale the point cloud.

Random Jittering: Add small random noise to point coordinates.

Random Dropout: Randomly remove a small percentage of points.

Why: Simulates variations in scanning conditions and object poses, making the model more robust and less prone to overfitting to specific scanner viewpoints or incomplete data. This can implicitly help the model learn to "fill in" missing parts better.

### 6. Feature Engineering (Optional but Potentially Powerful)
Problem Addressed: Providing the model with richer information beyond just XYZ coordinates.

Method:

Compute Normals: Estimate the surface normal vector for each point.

Estimate Curvature: Calculate local curvature (e.g., principal curvatures).

Local Density: Provide information about how dense the points are in a local neighborhood.

Why: Geometric features like normals and curvature are highly discriminative for shapes (e.g., pipes have high curvature in one direction, walls are flat). This can significantly boost segmentation accuracy, especially for thin structures.

Considerations for Model Selection (Re-emphasized):
Large-Scale Models: Given the point counts (millions per scan), you'll need models designed for large-scale point clouds, such as RandLA-Net or sparse convolutional networks (e.g., those using MinkowskiEngine). PointNet++ might be too slow or memory-intensive for direct application on full scans without aggressive downsampling or a hierarchical approach.

Robustness to Sparsity/Occlusion: The chosen model should ideally have mechanisms to handle varying point densities and occlusions. Local feature aggregation and contextual learning are important.

In [38]:
# --- Data Loading Function (Reused from previous successful script) ---
def load_station_data(station_id, ply_folder_path, dataset_root_path, global_labels_path, map_file_path):
    """
    Loads point cloud and corresponding labels for a given station ID and data split.
    """
    ply_file_name = f'SCAN_{station_id}.ply'
    ply_file_path = os.path.join(ply_folder_path, ply_file_name)

    # Load global labels (assuming already loaded once for efficiency if called repeatedly)
    # For this function, we'll load it each time for self-containment, but in a full pipeline,
    # you'd load all_labels once at the start.
    try:
        global_labels_df = pd.read_csv(global_labels_path, dtype={'class': np.int64})
        all_labels = global_labels_df['class'].values
    except Exception as e:
        print(f"Error loading global labels CSV: {e}")
        return None, None

    # Load map file
    try:
        map_df = pd.read_csv(map_file_path, header=None, names=['Station_index', 'index_start', 'index_end'],
                             dtype={'Station_index': np.int64, 'index_start': np.int64, 'index_end': np.int64})
        station_info = map_df[map_df['Station_index'] == station_id]
        if station_info.empty:
            print(f"Station ID {station_id} not found in map file: {map_file_path}")
            return None, None
        index_start = station_info['index_start'].iloc[0]
        index_end = station_info['index_end'].iloc[0]
    except Exception as e:
        print(f"Error loading map file or finding station info: {e}")
        return None, None

    # Extract labels for the specific station
    total_labels_count = len(all_labels)
    if index_end >= total_labels_count or index_start < 0 or index_end < index_start:
        print(f"Label index out of bounds or invalid for Station {station_id}: [{index_start}:{index_end+1}] "
              f"vs total labels {total_labels_count}. Skipping.")
        return None, None
    labels_for_station = all_labels[index_start : index_end + 1]

    # Load PLY data using plyfile
    try:
        plydata = PlyData.read(ply_file_path)
        points_element = plydata['points']
        points = np.vstack([points_element['x'], points_element['y'], points_element['z']]).T

        colors = None
        if 'rgb' in plydata:
            rgb_element = plydata['rgb']
            if 'r' in rgb_element.properties and 'g' in rgb_element.properties and 'b' in rgb_element.properties:
                colors = np.vstack([rgb_element['r'], rgb_element['g'], rgb_element['b']]).T
                if colors.max() > 1.0:
                    colors = colors / 255.0

        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        if colors is not None:
            pcd.colors = o3d.utility.Vector3dVector(colors)

        if not pcd.has_points():
            print(f"PLY file '{ply_file_name}' loaded but contains no points.")
            return None, None

    except Exception as e:
        print(f"Error loading PLY file '{ply_file_name}': {e}")
        return None, None

    # Final length check
    if len(pcd.points) != len(labels_for_station):
        print(f"Mismatch: Points ({len(pcd.points)}) != Labels ({len(labels_for_station)}) for Station {station_id}. Skipping.")
        return None, None

    return pcd, labels_for_station

In [41]:
# --- Preprocessing Functions ---

def remove_outliers(pcd, nb_neighbors=20, std_ratio=2.0, display_result=False):
    """
    Removes statistical outliers from the point cloud.
    :param pcd: Open3D PointCloud object.
    :param nb_neighbors: Number of neighbors to consider for each point.
    :param std_ratio: Standard deviation ratio.
    :param display_result: If True, visualize the point cloud after outlier removal.
    :return: Cleaned Open3D PointCloud object.
    """
    print(f"\n--- Outlier Removal (Statistical) ---")
    print(f"Original points: {len(pcd.points)}")
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    inlier_pcd = pcd.select_by_index(ind)
    outlier_pcd = pcd.select_by_index(ind, invert=True) # Optional: visualize outliers
    print(f"Points after outlier removal: {len(inlier_pcd.points)}")
    print(f"Removed {len(outlier_pcd.points)} outliers.")

    if display_result:
        # Color outliers red for visualization
        outlier_pcd.paint_uniform_color([1, 0, 0])
        # Color inliers green for visualization
        # inlier_pcd.paint_uniform_color([0, 1, 0]) # Commented out to preserve semantic colors later
        print("Visualizing inliers (original color) and outliers (red). Close window to continue.")
        o3d.visualization.draw_geometries([inlier_pcd, outlier_pcd])
    
    return inlier_pcd, ind # Return inlier indices to filter labels as well

In [44]:
def normalize_and_center(pcd, display_result=False):
    """
    Centers the point cloud and scales it to fit within a unit sphere.
    :param pcd: Open3D PointCloud object.
    :param display_result: If True, visualize the normalized point cloud.
    :return: Normalized Open3D PointCloud object.
    """
    print(f"\n--- Normalization and Centering ---")
    # Get points as NumPy array
    points = np.asarray(pcd.points)

    # 1. Center the point cloud
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    print(f"Centered point cloud by moving centroid to origin: {centroid}")

    # 2. Scale to unit sphere
    # Find the maximum distance from the origin
    max_distance = np.max(np.linalg.norm(points_centered, axis=1))
    if max_distance > 0:
        points_normalized = points_centered / max_distance
        print(f"Scaled point cloud to fit within unit sphere (max distance: {max_distance:.4f})")
    else:
        points_normalized = points_centered # Already at origin, no scaling needed
        print("Point cloud is a single point or empty, no scaling performed.")

    # Update the point cloud object
    pcd.points = o3d.utility.Vector3dVector(points_normalized)

    if display_result:
        print("Visualizing normalized and centered point cloud. Close window to continue.")
        o3d.visualization.draw_geometries([pcd])
    
    return pcd

In [46]:
def downsample_voxel_grid(pcd, voxel_size=0.05, display_result=False):
    """
    Downsamples the point cloud using a voxel grid.
    :param pcd: Open3D PointCloud object.
    :param voxel_size: Size of the voxel. Larger size means more aggressive downsampling.
                       Typical values: 0.01m to 0.1m for industrial scenes.
    :param display_result: If True, visualize the downsampled point cloud.
    :return: Downsampled Open3D PointCloud object.
    """
    print(f"\n--- Voxel Grid Downsampling ---")
    print(f"Original points: {len(pcd.points)}")
    downsampled_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    print(f"Points after downsampling (voxel size {voxel_size}m): {len(downsampled_pcd.points)}")

    if display_result:
        print("Visualizing downsampled point cloud. Close window to continue.")
        o3d.visualization.draw_geometries([downsampled_pcd])
    
    return downsampled_pcd

In [50]:
def compute_normals(pcd, radius=0.1, max_nn=30, display_result=False):
    """
    Estimates surface normals for each point.
    :param pcd: Open3D PointCloud object.
    :param radius: Search radius for neighbors.
    :param max_nn: Maximum number of neighbors to consider.
    :param display_result: If True, visualize point cloud with normals.
    :return: PointCloud object with normals.
    """
    print(f"\n--- Computing Normals ---")
    # Ensure point cloud has a KDTree for neighbor search
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    # Orient normals consistently (optional but good practice)
    pcd.orient_normals_to_align_with_direction(orientation_reference=np.array([0., 0., 1.])) # Align with Z-axis
    print(f"Normals computed for {len(pcd.points)} points.")

    if display_result:
        print("Visualizing point cloud with normals. Close window to continue.")
        # Normals are visualized as small lines extending from points
        o3d.visualization.draw_geometries([pcd], point_show_normal=True)
    
    return pcd

In [52]:
def augment_data(points, labels, rotation_range=(-np.pi/12, np.pi/12), scale_range=(0.8, 1.2), jitter_std=0.01):
    """
    Applies random rotation, scaling, and jittering to point cloud data.
    This function should be used ONLY for TRAINING DATA.
    :param points: NumPy array of shape (N, 3) for XYZ coordinates.
    :param labels: NumPy array of shape (N,) for class labels.
    :param rotation_range: Tuple (min_angle, max_angle) in radians for Z-axis rotation.
    :param scale_range: Tuple (min_scale, max_scale) for uniform scaling.
    :param jitter_std: Standard deviation for Gaussian noise jittering.
    :return: Augmented points and labels.
    """
    print(f"\n--- Applying Data Augmentation ---")
    
    # 1. Random Rotation (around Z-axis)
    angle = np.random.uniform(rotation_range[0], rotation_range[1])
    cosval = np.cos(angle)
    sinval = np.sin(angle)
    rotation_matrix = np.array([[cosval, -sinval, 0],
                                [sinval, cosval, 0],
                                [0, 0, 1]])
    points = np.dot(points.reshape(-1, 3), rotation_matrix)

    # 2. Random Scaling
    scale = np.random.uniform(scale_range[0], scale_range[1])
    points = points * scale

    # 3. Random Jittering
    jitter = np.random.normal(0, jitter_std, points.shape)
    points = points + jitter

    print("Applied random rotation, scaling, and jittering.")
    return points, labels


In [56]:
# --- Main Execution Flow for Preprocessing Demonstration ---

print(f"--- Starting Preprocessing Demo for Station {station_id_to_visualize}  ---")

# 1. Load the raw point cloud and labels
pcd_raw, labels_raw = load_station_data(station_id_to_visualize, ply_folder_path,
                                        dataset_root_path, global_labels_path, map_file_path)

if pcd_raw is None:
    print("Failed to load initial data. Exiting.")
    sys.exit(1)

print(f"\n--- Raw Data Loaded ---")
print(f"Points: {len(pcd_raw.points)}, Labels: {len(labels_raw)}")

# Apply semantic colors for initial visualization
colors_initial = np.zeros((len(labels_raw), 3))
for i, label_id in enumerate(labels_raw):
    colors_initial[i] = CLASS_COLORS.get(int(label_id), [0.8, 0.8, 0.8])
pcd_raw.colors = o3d.utility.Vector3dVector(colors_initial)

print("Visualizing raw data with semantic colors. Close window to continue.")
o3d.visualization.draw_geometries([pcd_raw])

--- Starting Preprocessing Demo for Station 4  ---

--- Raw Data Loaded ---
Points: 1654791, Labels: 1654791
Visualizing raw data with semantic colors. Close window to continue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not 

In [60]:
# 2. Outlier Removal
pcd_cleaned, inlier_indices = remove_outliers(pcd_raw, nb_neighbors=30, std_ratio=2.0, display_result=True)
# Filter labels to match the cleaned point cloud
labels_cleaned = labels_raw[inlier_indices]

# Re-apply semantic colors to the cleaned PCD for subsequent steps
colors_cleaned = np.zeros((len(labels_cleaned), 3))
for i, label_id in enumerate(labels_cleaned):
    colors_cleaned[i] = CLASS_COLORS.get(int(label_id), [0.8, 0.8, 0.8])
pcd_cleaned.colors = o3d.utility.Vector3dVector(colors_cleaned)



--- Outlier Removal (Statistical) ---
Original points: 1654791
Points after outlier removal: 1623601
Removed 31190 outliers.
Visualizing inliers (original color) and outliers (red). Close window to continue.


In [61]:
# 3. Normalization and Centering
pcd_normalized = normalize_and_center(pcd_cleaned, display_result=True)
# Note: Normalization only changes coordinates, not point count or labels.
# The labels_cleaned array is still valid for pcd_normalized.


--- Normalization and Centering ---
Centered point cloud by moving centroid to origin: [ 19838.9410635    1786.32816424 152488.63598938]
Scaled point cloud to fit within unit sphere (max distance: 10341.9251)
Visualizing normalized and centered point cloud. Close window to continue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [64]:
# 4. Downsampling (for training, you might choose a smaller voxel_size)
# For visualization, a slightly larger voxel_size can make it faster.
pcd_downsampled = downsample_voxel_grid(pcd_normalized, voxel_size=0.03, display_result=True)
# Downsampling changes point count, so need to re-map labels if you want to use them
# This is more complex than simple slicing. For now, we'll just show the downsampled geometry.
# In a real training pipeline, you'd ensure labels are correctly associated with downsampled points.
# Open3D's voxel_down_sample can return indices, but mapping labels is non-trivial for all methods.
# For a deep learning pipeline, you'd typically apply downsampling *before* you extract features/labels for batches.

# For demonstration, let's just visualize the downsampled point cloud without labels
# If you want labels on downsampled points, you need to ensure the downsampling method preserves label correspondence.
# For voxel_down_sample, the centroid of a voxel gets the label of the majority point in that voxel, or one point's label.
# This is often handled within the dataset loading for the ML framework.
print(f"\nNote: Labels are not directly re-associated with downsampled points in this demo for simplicity.")
print(f"In a full training pipeline, ensure labels are correctly propagated during downsampling.")



--- Voxel Grid Downsampling ---
Original points: 1623601
Points after downsampling (voxel size 0.03m): 1121
Visualizing downsampled point cloud. Close window to continue.

Note: Labels are not directly re-associated with downsampled points in this demo for simplicity.
In a full training pipeline, ensure labels are correctly propagated during downsampling.


In [67]:
# 5. Compute Normals (added as a feature)
# This is often done after downsampling if you want normals for the downsampled points.
pcd_with_normals = compute_normals(pcd_downsampled, radius=0.1, max_nn=30, display_result=True)
# Normals are stored as a property of the PCD object and would be passed to the model.

# --- Data Augmentation Demonstration (ONLY for training data) ---
print("\n--- Demonstrating Data Augmentation (for Training Data Only) ---")
print("This step is applied on the NumPy arrays of points and labels during training.")

# Get points and labels as NumPy arrays from the *cleaned* data for augmentation demo
# (or from the downsampled data if your model expects augmented downsampled points)
points_for_augmentation = np.asarray(pcd_cleaned.points)
labels_for_augmentation = labels_cleaned.copy() # Make a copy to avoid modifying original



--- Computing Normals ---
Normals computed for 1121 points.
Visualizing point cloud with normals. Close window to continue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 

--- Demonstrating Data Augmentation (for Training Data Only) ---
This step is applied on the NumPy arrays of points and labels during training.


In [68]:
# Apply augmentation
points_augmented, labels_augmented = augment_data(
    points_for_augmentation, labels_for_augmentation,
    rotation_range=(-np.pi/12, np.pi/12), # +/- 15 degrees
    scale_range=(0.9, 1.1),             # +/- 10% scale
    jitter_std=0.005                    # Small jitter
)

# Visualize augmented data (optional, for verification)
pcd_augmented = o3d.geometry.PointCloud()
pcd_augmented.points = o3d.utility.Vector3dVector(points_augmented)

# Re-apply semantic colors to augmented PCD
colors_augmented = np.zeros((len(labels_augmented), 3))
for i, label_id in enumerate(labels_augmented):
    colors_augmented[i] = CLASS_COLORS.get(int(label_id), [0.8, 0.8, 0.8])
pcd_augmented.colors = o3d.utility.Vector3dVector(colors_augmented)

print("Visualizing augmented point cloud. Close window to continue.")
o3d.visualization.draw_geometries([pcd_augmented])

print("\n--- Preprocessing Demo Complete ---")
print("Remember to integrate these steps into your full data loading pipeline for training and inference.")
print("Class imbalance handling (weighted loss, sampling) will be part of your model's training loop.")


--- Applying Data Augmentation ---
Applied random rotation, scaling, and jittering.
Visualizing augmented point cloud. Close window to continue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 

--- Preprocessing Demo Complete ---
Remember to integrate these steps into your full data loading pipeline for training and inference.
Class im

# Next Steps: Model Development & Training
## 1. Prepare Data for Model Input (Custom Dataset & DataLoader)
Your preprocessing functions are great, but a deep learning model needs data in a specific, batch-friendly format.

Create a Custom Dataset Class (e.g., EDFPointCloudDataset):

This class will inherit from PyTorch's torch.utils.data.Dataset (or TensorFlow's equivalent).

Its __init__ method will:

Load the ytrain_map_ind_station.csv (and ytest_map_ind_station.csv) to get the list of station IDs and their index ranges.

Load the ytrain_i9bpfD4.csv (global labels) once into memory.

Its __len__ method will return the total number of individual points or (more practically for large scans) the total number of patches if you're processing in patches.

Its __getitem__(self, idx) method is the most important:

Given an idx, it will determine which SCAN_X.ply file and corresponding label segment to load.

It will load the raw point cloud and labels using your load_station_data function.

It will apply the chosen preprocessing steps (outlier removal, normalization, optional downsampling, normal computation).

Crucially, for training data, it will apply Data Augmentation.

It will return the processed points (and optionally normals, colors) and their corresponding labels as NumPy arrays or PyTorch/TensorFlow tensors.

Handling Large Scans / Patching: For very large scans (like SCAN_8 with 3.3 million points), you often don't feed the entire scan to the GPU at once. Instead, you extract smaller, overlapping "patches" or "blocks" from the scan. Your __getitem__ would then return a patch, and its corresponding labels. This is a common strategy for large point cloud datasets.

Create a DataLoader (e.g., torch.utils.data.DataLoader):

This will wrap your custom dataset.

It handles batching (grouping multiple processed point clouds/patches together).

It handles shuffling (randomizing the order of samples for training).

It handles multiprocessing (loading data in parallel to keep the GPU busy).

## 2. Choose a 3D Segmentation Model Architecture
Based on your observations (large-scale data, class imbalance, complex industrial environment, missing sections/occlusions), here are the recommended model types:

Primary Recommendation: RandLA-Net

Why: It's designed for efficiency on large-scale point clouds by using random sampling and local feature aggregation. This directly addresses your large point counts and helps with varying densities. It's a strong candidate for industrial scenes.

Resources: You can find PyTorch or TensorFlow implementations on GitHub (e.g., QingyongHu/RandLA-Net).

Alternative/Advanced: Sparse Convolutional Networks (e.g., MinkowskiEngine-based)

Why: These models operate on sparse voxelized representations, which are very memory-efficient and can be highly accurate for detailed scenes. They are excellent for capturing fine structures like pipes.

Complexity: Can be more complex to set up than PointNet-like architectures.

Resources: StanfordVL/MinkowskiEngine.

Baseline (if others are too complex initially): PointNet++

Why: A foundational model that introduced hierarchical feature learning. It's good for understanding the basics.

Limitations: Might struggle with very large point clouds without significant downsampling or complex patching strategies.

Resources: charlesq34/pointnet2 (TensorFlow), many PyTorch re-implementations.

## 3. Implement the Model
Once you choose an architecture, you'll implement it using your chosen deep learning framework (PyTorch or TensorFlow).

Define the Network: Create the neural network layers (e.g., convolutional layers, pooling layers, fully connected layers) as specified by the chosen architecture.

Input Features: Your model will take as input:

XYZ coordinates (normalized).

Optionally, RGB colors (normalized).

Optionally, Intensity (normalized, if you decide to use it).

Optionally, Normals (computed in preprocessing).

Optionally, other engineered features (e.g., local density).

Output: The model will output a prediction for each point, typically a logit or probability score for each of your 10 semantic classes.

## 4. Define Loss Function and Optimizer
Loss Function:

Weighted Cross-Entropy Loss: This is CRITICAL for addressing the severe class imbalance you observed. You'll assign higher weights to the minority classes (e.g., Guardrails, Ladders, Beams) and lower weights to the majority classes (Pipping, Civils). This forces the model to pay more attention to correctly classifying the rare but important objects.

How to calculate weights: Weights are typically inversely proportional to the class frequency. You'll need to calculate the point counts for all classes across your entire training dataset to get accurate weights.

Optimizer:

Adam or AdamW: Common and effective optimizers for deep learning.

## 5. Set Up the Training Loop
This is the core iterative process where the model learns.

Epochs: Number of times the model sees the entire training dataset.

Batch Size: Number of point clouds/patches processed at once by the GPU.

Learning Rate Scheduler: Adjusts the learning rate during training (e.g., reduce on plateau, cosine annealing) to improve convergence.

Forward Pass: Feed batches of input data through the model to get predictions.

Loss Calculation: Compute the weighted cross-entropy loss between predictions and ground truth labels.

Backward Pass & Optimization: Compute gradients and update model weights.

Validation: Periodically (e.g., every few epochs), evaluate the model on a separate validation set to monitor performance and prevent overfitting.

Model Checkpointing: Save the model weights (and optimizer state) at regular intervals or when validation performance improves.

## 6. Evaluation Metrics
Beyond visual inspection, you need quantitative metrics to assess your model's performance.

Intersection over Union (IoU) / Jaccard Index:

Calculated per class: (True Positives) / (True Positives + False Positives + False Negatives).

Mean IoU (mIoU): The average IoU across all classes. This is the primary metric for semantic segmentation.

Why: Directly measures the overlap between predicted and ground truth segments.

Overall Accuracy: (Total Correctly Classified Points) / (Total Points).

Caution: Can be misleading with high class imbalance (a model predicting only 'Pipping' and 'Civils' could still have high overall accuracy).

F1-Score (per class): Harmonic mean of precision and recall. Useful for understanding performance on individual classes, especially minority ones.

